## A. 
Data exploration
Explore the three datasets. Use your judgment as a human being to describe
what makes you able to tell spam apart from ham. Also explore the differences between the two classes of ham (easy and hard ham). What makes them different? Write approximately one paragraph.

1. ##### Difference between Ham and Spam
    Spam contains a lot of clickbaity titles with links that looks more than suspicious. Very sensational language and the senders adress looks very auto-generated for example: sabrina@mx3.1premio.com.
    Ham on the other hand is more legit and the emails looks like they are sent from either real people or real companies with a newsletter and you get the choice to unsubscribe or not.

2. ##### Difference between Hard and Easy Ham
    Easy ham seems to be mostly conversations between real life people, not alot of code, mostly natural speaking language
    Hard Ham contains more code, and contains mostly of commercials which makes it hard to judge if it's actually wanted by the reciever or not.

## B.
Perform an appropriate train-test split on the each of the datasets. We will use
the training sets to train a classifier, and evaluate its performance against the
test sets.

In [6]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split

# Setting the random state to 1 to ensure reproducibility
RANDOM_STATE = 1

# Creating lists and dicts to parse the text files
texts: list[str] = []
labels: list[str] = []
easy_ham_dir_path = os.getcwd() + "/20021010_easy_ham/easy_ham/"
hard_ham_dir_path = os.getcwd() + "/20021010_hard_ham/hard_ham/"
spam_dir_path = os.getcwd() + "/20021010_spam/spam/"

# Parsing the text files into lists
for file_name in os.listdir(easy_ham_dir_path):
    f = open(easy_ham_dir_path+file_name, "rb")
    text = f.read()
    texts.append(text)
    labels.append("easy_ham")
    f.close()

for file_name in os.listdir(hard_ham_dir_path):
    f = open(hard_ham_dir_path+file_name, "rb")
    text = f.read()
    texts.append(text)
    labels.append("hard_ham")
    f.close()

for file_name in os.listdir(spam_dir_path):
    f = open(spam_dir_path+file_name, "rb")
    text = f.read()
    texts.append(text)
    labels.append("spam")
    f.close()

# Making the lists series for nicer printing
texts = pd.Series(texts)
labels = pd.Series(labels)

# Make a simple train-test split using 25% of the data for testing as this was recommended in last thursday's lecture
x_train, x_test, y_train, y_test = train_test_split(texts, labels, test_size=0.25, random_state=RANDOM_STATE)

print(x_train)
print(y_train)

285     b"From spamassassin-talk-admin@lists.sourcefor...
2692    b'From rpm-list-admin@freshrpms.net  Wed Oct  ...
3210    b'From hezf855@framesetup.com  Thu Aug 29 10:5...
1603    b'From fork-admin@xent.com  Sun Sep 22 14:11:4...
91      b'From rpm-list-admin@freshrpms.net  Tue Sep 1...
                              ...                        
2763    b'Return-Path: <Online#3.20483.52-WfysgP14zsqz...
905     b"From rpm-list-admin@freshrpms.net  Mon Oct  ...
1096    b"From rssfeeds@jmason.org  Thu Sep 26 16:43:2...
235     b"From rssfeeds@jmason.org  Tue Oct  1 10:36:3...
1061    b'From fork-admin@xent.com  Tue Sep 24 10:49:2...
Length: 2476, dtype: object
285     easy_ham
2692    hard_ham
3210        spam
1603    easy_ham
91      easy_ham
          ...   
2763    hard_ham
905     easy_ham
1096    easy_ham
235     easy_ham
1061    easy_ham
Length: 2476, dtype: object
